## Deliverable 2. Create a Customer Travel Destinations Map.

In [41]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np
# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [42]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Bluff,NZ,2021-10-31 02:26:26,-46.6000,168.3333,49.51,76,100,28.23,overcast clouds
1,1,Uribia,CO,2021-10-31 02:26:26,11.7139,-72.2660,78.33,83,100,5.41,overcast clouds
2,2,Nalut,LY,2021-10-31 02:26:26,30.3333,10.8500,65.52,40,29,4.50,scattered clouds
3,3,Dingle,PH,2021-10-31 02:26:26,10.9995,122.6711,82.63,78,81,15.41,broken clouds
4,4,Whitehorse,CA,2021-10-31 02:26:27,60.7161,-135.0538,36.37,69,75,13.00,broken clouds


In [43]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
max_temp = float(input("What is the maximum tempurature you would like to experience? "))
min_temp = float(input("What is the minimum tempurature you would like to experience? "))

What is the maximum tempurature you would like to experience? 85
What is the minimum tempurature you would like to experience? 55


In [44]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]

#preferred_cities_df.isnull().sum().dropna()
preferred_cities_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Uribia,CO,2021-10-31 02:26:26,11.7139,-72.2660,78.33,83,100,5.41,overcast clouds
2,2,Nalut,LY,2021-10-31 02:26:26,30.3333,10.8500,65.52,40,29,4.50,scattered clouds
3,3,Dingle,PH,2021-10-31 02:26:26,10.9995,122.6711,82.63,78,81,15.41,broken clouds
9,9,Tias,ES,2021-10-31 02:26:28,28.9611,-13.6450,65.95,82,0,10.36,clear sky
10,10,Hualmay,PE,2021-10-31 02:26:28,-11.0964,-77.6139,63.32,82,96,7.40,overcast clouds
12,12,Rikitea,PF,2021-10-31 02:26:28,-23.1203,-134.9692,74.59,87,100,21.83,moderate rain
13,13,Saint George,US,2021-10-31 02:22:37,37.1041,-113.5841,69.35,47,1,1.01,clear sky
16,16,Padang,ID,2021-10-31 02:26:30,-0.9492,100.3543,78.85,81,99,1.43,overcast clouds
17,17,Busselton,AU,2021-10-31 02:26:30,-33.6500,115.3333,66.04,63,97,3.49,overcast clouds
18,18,Concordia,AR,2021-10-31 02:26:30,-31.3930,-58.0209,75.78,64,0,1.99,clear sky


In [45]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                7
Date                   0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [46]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df.count()

City_ID                408
City                   408
Country                408
Date                   408
Lat                    408
Lng                    408
Max Temp               408
Humidity               408
Cloudiness             408
Wind Speed             408
Current Description    408
dtype: int64

In [47]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Uribia,CO,78.33,overcast clouds,11.7139,-72.2660,
2,Nalut,LY,65.52,scattered clouds,30.3333,10.8500,
3,Dingle,PH,82.63,broken clouds,10.9995,122.6711,
9,Tias,ES,65.95,clear sky,28.9611,-13.6450,
10,Hualmay,PE,63.32,overcast clouds,-11.0964,-77.6139,
12,Rikitea,PF,74.59,moderate rain,-23.1203,-134.9692,
13,Saint George,US,69.35,clear sky,37.1041,-113.5841,
16,Padang,ID,78.85,overcast clouds,-0.9492,100.3543,
17,Busselton,AU,66.04,overcast clouds,-33.6500,115.3333,
18,Concordia,AR,75.78,clear sky,-31.3930,-58.0209,


In [48]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        hotel_df.loc[index, "Hotel Name"] = np.nan
        print("Hotel not found... skipping.")

        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [53]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.count()
clean_hotel_df =hotel_df.dropna()
clean_hotel_df.count()

City                   379
Country                379
Max Temp               379
Current Description    379
Lat                    379
Lng                    379
Hotel Name             379
dtype: int64

In [57]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [58]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [59]:
# 11a. Add a marker layer for each city to the map. 
locations = clean_hotel_df[["Lat", "Lng"]]
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))